In [19]:
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [20]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                txt_data = file.read().split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)
    
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(all_texts_extend)
    
    print(all_texts)
    print("-----------------")
    print(all_texts_extend)
    print("-----------------")

    
    all_sequences=[]
    for i in range(len(all_texts)):
        #print(tokenizer.texts_to_sequences(all_texts[i]))
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))
        
    print("ALL SEQUENCES,",all_sequences)
    print("-----------------")
    
    
    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'
    
    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)

    #new_array=np.array(all_padded_sequences,dtype=object)

    
    return all_padded_sequences, tokenizer.word_index

In [21]:
def add_zero_arrays(arr):
    while len(arr) < 360:
        arr = np.concatenate((arr, np.zeros((1, 20), dtype=np.int32)))
    return arr

In [ ]:
directory_path = '.'
padded_sequences, word_index = process_text_files(directory_path)

for i in range(len(padded_sequences)):
    print(i,padded_sequences[i])
    
print("-----------------")

print(word_index)

print("-----------------")

#result_list = [add_zero_arrays(arr) for arr in padded_sequences]

#print(result_list)
# Printing the result
#for arr in result_list:
    #print(arr)

In [ ]:
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re

def process_text_files(directory):
    all_texts = []
    all_texts_extend = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Remove the tags
                content = re.sub(r'<start_".*?">', '', content)
                content = re.sub(r'<end_".*?">', '', content)
                txt_data = content.split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)
    
    tokenizer = Tokenizer(char_level=True, lower=False)
    tokenizer.fit_on_texts(all_texts_extend)
    print(tokenizer.word_index)
    
    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))
        
    max_total_word_length=3
    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'
    
    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)
    
    return all_padded_sequences, tokenizer.word_index

directory_path = '.'
padded_sequences, word_index = process_text_files(directory_path)

for i in range(len(padded_sequences)):
    np.set_printoptions(threshold=np.inf)
    print(padded_sequences[i])
    np.set_printoptions(threshold=1000)

print("-----------------")
print(word_index)
print("-----------------")

padded_arrays = []
for arr in padded_sequences:
    pad_width = ((0, 384 - len(arr)), (0, 0))
    padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
    padded_arrays.append(padded_arr)

print(padded_arrays)


In [1]:
import os

wordsList = []
wordsIndextList = []


label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]

 

def one_hot_encode_index_list():
    oneHotEncodedList = []
    unique_labels_len = len(sorted(set(label_list2)))
    for i in wordsIndextList:
        bitlist = list(0 for i in range(unique_labels_len))
        bitlist[i] = 1
        oneHotEncodedList.append(bitlist)
    return oneHotEncodedList


def splitfunction(text:str):
    words = text.split()
    for word in words:
        wordsList.append(word)
        wordsIndextList.append(1)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag    
    return None



def tagingWords():
    control = None
    counter = 0
    for word in wordsList:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndextList[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndextList[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1  


def oneHotEncodedFunction(texts):
    wordsList.clear
    wordsIndextList.clear
    splitfunction(texts)
    tagingWords()
    return one_hot_encode_index_list()




folder_path = "./training-data"
file_list = os.listdir(folder_path)
y_train = []

for file_name in file_list:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            oneHotEncodedList = oneHotEncodedFunction(text)
            y_train.append(oneHotEncodedList)
            

In [45]:
# LSTM Modeli

import tensorflow 
from tensorflow.keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten
from keras.models import Model

batch_size = 16
max_sequence_length = 384
max_word_length = 20
embedding_size = 128
lstm_units = 256
num_classes = 14

input_shape = (max_sequence_length, max_word_length)
input_layer = Input(shape=(max_sequence_length,max_word_length), batch_size=16)
print(input_layer.shape)


(16, 384, 20)


In [42]:

char_embedding = Embedding(input_dim=max_word_length, output_dim=embedding_size)(input_layer)
print(char_embedding.shape)


(16, 384, 20, 128)


In [ ]:
#reshaped_embedding = Reshape(target_shape=(batch_size * max_sequence_length, max_word_length, embedding_size))(char_embedding)
#reshaped_embedding = tensorflow.reshape(char_embedding, (batch_size * max_sequence_length, max_word_length, embedding_size))
#print(reshaped_embedding.shape)

char_embedding_tensor = tensorflow.convert_to_tensor(char_embedding)
reshaped_embedding = Reshape(target_shape=(batch_size * max_sequence_length, max_word_length, embedding_size))(char_embedding_tensor)

In [ ]:
char_lstm, _, _ = LSTM(units=lstm_units, return_sequences=True, return_state=True)(reshaped_embedding)
print(char_lstm)

char_lstm2, _, _ = LSTM(units=lstm_units, return_sequences=True, return_state=True)(char_lstm)
print(char_lstm2)

In [ ]:
reshaped = Reshape(target_shape=(batch_size,max_sequence_length, max_word_length)) (char_lstm2)
print(reshaped.shape)

In [ ]:
word_lstm1 = LSTM(units=lstm_units, return_sequences=True)(reshaped)
print(word_lstm1)
word_lstm2 = LSTM(units=lstm_units, return_sequences=True)(word_lstm1)
print(word_lstm2)

In [ ]:
classification_output = Dense(units=num_classes, activation='softmax')(word_lstm2)
print(word_lstm2)

In [ ]:
model = Model(inputs=input_layer, outputs=[classification_output, char_lstm])


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()